In [ ]:
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import numpy as np
import plotnine as p9
import matplotlib.pyplot as plt
import graphviz
import seaborn as sn
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, validation_curve, learning_curve, train_test_split
from sklearn.metrics import roc_curve, auc, mean_squared_error, plot_confusion_matrix
from sklearn.linear_model import Lasso
from math import *
from statistics import *
from tabulate import tabulate

In [ ]:
#Read in the data set
data = pd.read_csv('../Data_and_Documentation/37633-0001-Data.csv')

In [ ]:
#Drop all observations where dependent variable = -1 (missing)
data = data.loc[data['Q16_4'] != -1]

In [ ]:
#Subset only the variables of interest
data = data[['DEM_AGE_LONG','DEM_GENDER','DEM_REGION','DEM_EDUCATION_SHORT','DEM_MARITALSTATUS_LONG','DEM_HOUSEHOLDSIZE',
 'DEM_FAMILYINCOME_SHORT','DEM_EMPLOYMENT','DEM_ETHNICITY_LONG','Q11A','Q11B','Q15A_ATGE','Q15A_AHC','Q15A_AHI','Q15A_CE','Q15A_D',
           'Q15A_ER','Q15A_GBM','Q15A_HITH','Q15A_L','Q15A_PCB','Q15A_TBSTLI','Q16_4']]
data.head(10)

In [ ]:
#Gender is originally coded as 1 = Male, 2 = Female. Subtract 1 from this variable to make it binary
data['DEM_GENDER'] = [x - 1 for x in data['DEM_GENDER']]

In [ ]:
#This merges the two forms of Q11 together. 0 = personal factors, 1 = environmental factors
data['Q11'] = data['Q11A'].combine_first(data['Q11B'])

In [ ]:
#Drop the superfluous columns, subtract 1 for the same reason as the gender variable
data = data.drop(columns = ['Q11A','Q11B'])
data['Q11'] = [x - 1 for x in data['Q11']]

In [ ]:
#Convert the categorical ones into dummies
data = pd.get_dummies(data, columns=['DEM_REGION', 'DEM_EDUCATION_SHORT', 
                                     'DEM_MARITALSTATUS_LONG', 'DEM_EMPLOYMENT', 
                                     'DEM_ETHNICITY_LONG'], drop_first=True)
data.head(5)

In [ ]:
#Print the column names to make it easier to reference for renaming in the next cell
#data.columns

In [ ]:
#Rename dummies for clarity
data = data.rename(columns = {'DEM_AGE_LONG': 'Age', 'DEM_GENDER': 'Female', 'DEM_HOUSEHOLDSIZE': "HHoldSize",
       'DEM_FAMILYINCOME_SHORT': 'HHIncome','DEM_REGION_1': 'R_NEast',
       'DEM_REGION_2': 'Midwest', 'DEM_REGION_3': 'South', 'DEM_REGION_4': 'West',
       'DEM_EDUCATION_SHORT_2': 'HighSchool',
       'DEM_EDUCATION_SHORT_3': 'Assoc_Degree', 'DEM_EDUCATION_SHORT_4': '4YearDegree', 
       'DEM_MARITALSTATUS_LONG_2': 'Separated', 'DEM_MARITALSTATUS_LONG_3': 'Divorced',
       'DEM_MARITALSTATUS_LONG_4': 'Widowed', 'DEM_MARITALSTATUS_LONG_5': 'NeverMarried',
       'DEM_EMPLOYMENT_2': 'SelfEmployed', 'DEM_EMPLOYMENT_3': 'TempLayoff', 'DEM_EMPLOYMENT_4': 'Unemployed',
       'DEM_EMPLOYMENT_5': 'Retired', 'DEM_EMPLOYMENT_6': 'Disabled', 'DEM_EMPLOYMENT_7': "Other_Nonworking",
       'DEM_ETHNICITY_LONG_2': 'Black', 'DEM_ETHNICITY_LONG_3': 'Hispanic', 'DEM_ETHNICITY_LONG_4': 'AAPI',
       'DEM_ETHNICITY_LONG_5': 'Indigenous', 'DEM_ETHNICITY_LONG_6': 'Other_Races'})

In [ ]:
data.isna().sum()

In [ ]:
data = data.dropna()

X = data.drop(columns = ['Q16_4'], axis = 1)
y = data['Q16_4']

scaler = StandardScaler()
Xsc = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

Xtrain, Xtest, ytrain, ytest = train_test_split(Xsc, y, test_size=0.25, random_state=10)

In [ ]:
lasso = Lasso(alpha = 0.5)
lasso.fit(Xtrain, ytrain)
lasso.score(Xtest, ytest)

In [ ]:
output = pd.DataFrame({'Variable': Xsc.columns, 'Lasso Coef': lasso.coef_}).sort_values(['Lasso Coef'], ascending = False)
print(output.to_latex(index = False))
#print(tabulate(output, tablefmt="pipe", headers="keys"))

In [ ]:
np.percentile(data['Q16_4'], [33,66])

In [ ]:
X = data.drop(columns = ['Q16_4'], axis = 1)
y = data['Q16_4']

In [ ]:
y = pd.DataFrame(y)
conditions = [(y <= 33), (y > 66)]
choices = [0,2]
y['cat_col'] = np.select(conditions, choices, default = 1)


In [ ]:
y = y['cat_col']

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=10)

In [ ]:
#Also check validation curve for max_depth between 1 and 10
tree = DecisionTreeClassifier()
train_scores, test_scores = validation_curve(tree, X, y, param_name='min_samples_leaf',
                                            param_range=np.arange(70,80))

In [ ]:
#Find training and test means
train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

In [ ]:
#Plot validation curve
plt.plot(np.arange(70,80), train_mean, label='Train')
plt.plot(np.arange(70,80), test_mean, label='Test')
plt.xlabel('Value')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Note: max_depth best test score = 5, min_samples_leaf = 80

In [ ]:
#Initialize new classifier and fit
tree = DecisionTreeClassifier(min_samples_leaf = 80, max_depth = 5)
tree.fit(Xtrain, ytrain)

In [ ]:
#cross validate mean accuracy
cross_val_score(tree, Xtest, ytest, cv=8).mean()

In [ ]:
#plot confusion matrix and output
plot_confusion_matrix(tree, Xtest, ytest)
#plt.savefig('dt_cm.png')

In [ ]:
#Output a .dot file for the tree
dot_data = export_graphviz(tree, out_file='tree.dot', feature_names=Xtrain.columns, rotate = True)
graph = graphviz.Source(dot_data)
#graph.render("data")

Run the following line in the command line to produce a .png:

dot -Tpng tree.dot -o tree.png

In [ ]:
#output tree feature importances
output_tree = pd.DataFrame({'Variable': X.columns, 'Tree_Feat': tree.feature_importances_}).sort_values(['Tree_Feat'], ascending = False)
output_tree

Now for the random forest

In [ ]:
#Initialize the classifier, run the max_depth with 1-10, run the n_estimators with 100-1000 (increments of 100)
#NOTE: The n_estimators one will take a long time
forest = RandomForestClassifier()
train_scores, test_scores = validation_curve(forest, X, y, param_name='n_estimators',
                                            param_range=(100,200,300,400,500,600,700,800,900,1000))

In [ ]:
#Find training and test means
train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

In [ ]:
#Plot validation curve
plt.plot((100,200,300,400,500,600,700,800,900,1000), train_mean, label='Train')
plt.plot((100,200,300,400,500,600,700,800,900,1000), test_mean, label='Test')
plt.xlabel('Value')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

For random forest: max_depth best score = 7, n_estimators = 600

In [ ]:
#Initialize rf classifier, fit to training data
forest = RandomForestClassifier(max_depth = 7, n_estimators = 600)
forest.fit(Xtrain, ytrain)

In [ ]:
#cross validate mean accuracy
cross_val_score(forest, Xtest, ytest, cv = 8).mean()

In [ ]:
#plot confusion matrix, output to png
plot_confusion_matrix(forest, Xtest, ytest)
plt.savefig('rf_cm.png')

In [ ]:
#view RF feature importances
output_forest = pd.DataFrame({'Variable': X.columns, 'Forest_Feat': forest.feature_importances_}).sort_values(['Forest_Feat'])
output_forest

In [ ]:
#Merge tree and forest features into 1 table, output into latex form 
feat_import = output_tree.merge(output_forest, on = 'Variable')
print(feat_import.to_latex(index = False))

In [ ]:
#generate dataframe of predictions for tree an forest
predictions = pd.DataFrame({'tree_pred': tree.predict(Xtest), 'rf_pred': forest.predict(Xtest), 'true': ytest})
#find number of cases where predictions are different
predictions['same'] = np.where(predictions['tree_pred'] == predictions['rf_pred'], 0,1)
#find number of cases where predictions are the same and are accurate
predictions['correct'] = np.where((predictions['same'] == 0) & (predictions['tree_pred'] == predictions['true']),1,0)
predictions.head(10)

In [ ]:
# number of correct predictions
len(predictions) - predictions['correct'].sum()

In [ ]:
#percent of correct predictions
predictions['correct'].sum()/len(predictions)